In [3]:
from dotenv import load_dotenv
import os
import pickle

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()

True

In [4]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [5]:
# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [6]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [7]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [8]:
review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new  chain...


Tragedy at Sunset on the Beach is a powerful drama about a young couple living in a small beach town in the south. They have been together since they were teenagers and have recently had their first child together. Just before sunset, they decide to take a walk on the beach to celebrate their new addition to the family. 

But their joy and celebration is quickly cut short when they stumble upon a crime scene. A young woman has been brutally murdered and the police are currently investigating the scene. As the couple watches in horror, the detectives uncover evidence that the murderer is still on the loose and could be watching them from the shadows.

The couple is faced with a difficult decision; do they stay and help solve the mystery or do they flee to protect their own safety? As the sun sets, they must make a choice that could forever alter the course of their lives. Tragedy at Sunset on the Beach is a gripping story of love, loss, and the power of seco

In [10]:
print(review)



Tragedy at Sunset on the Beach is a captivating and emotionally powerful drama that will leave audiences breathless. The story follows a young couple living in a small beach town in the south, who have recently had their first child together. Just as they are celebrating their new addition to the family, they stumble upon a crime scene - a young woman has been brutally murdered and the police are investigating the scene.

The couple soon finds themselves face to face with a difficult decision, do they stay and help solve the mystery or do they flee to protect their own safety? The play examines the power of love in the face of fear and tragedy, as the couple grapple with the consequences of their decision.

The performances are outstanding, with each actor bringing a unique energy to their roles. The dialogue is poignant and powerful, and the tension builds slowly and steadily throughout the play. The set design is also excellent, immersing viewers in the atmosphere of the small beac

# Ordering

In [11]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

In [12]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

In [13]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [14]:
overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new  chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "\n\nTragedy at Sunset on the Beach is set in Victorian England, and tells the story of a young woman, Emma, and her forbidden love for a dashing stranger. Emma has lived an uneventful life in a small coastal town. One day, while out on the beach, she meets a mysterious stranger, who sweeps her off her feet with his passionate and captivating words.\n\nThough Emma's parents disapprove of their daughter's relationship with the stranger, they do not forbid it as they know how much she means to him. However, their happiness is short lived as the stranger is revealed to be a fugitive wanted by the authorities. Emma is forced to make a difficult decision - help the man she loves escape or save her family's good name.\n\nThe play follows Emma's tumultuous journey as she is forced to choose between her love and her family. As the play progresses, tragedy ensues and the sun sets on the beach, leaving the audi

In [17]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

llm = OpenAI(temperature=.7)
template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=template)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

memory = SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"})
overall_chain = SequentialChain(
    memory=memory,
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new  chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': 'Experience a powerful and heartbreaking story of love and loss this holiday season! On December 25th at 8pm PST, join us at the Theater in the Park for Tragedy at Sunset on the Beach. Our production captures the tragedy of this moment with powerful emotion and a set that evokes the beauty of the beach and the sadness of the moment. Don’t miss this touching reminder of the power of love! #TragedyAtSunsetOnTheBeach #LoveConquersAll'}

In [22]:
overall_chain({"title":"Cloud Summit", "era": "Taipei city"})



> Entering new  chain...

> Finished chain.


"\nCome join us at Theater in the Park on December 25th at 8pm PST to experience the heart-warming and thought-provoking story of Cloud Summit. Follow Chen Mei's journey of resilience and self-discovery as she navigates the turbulent waters of business and politics in the bustling city of Taipei. Don't miss this inspiring and empowering play - get your tickets now! #CloudSummit #Theater #Inspiration"